# Unsupervised Learning

### Loading Libraries

In [ ]:
# Numerical Computing
import numpy as np

# Data Manipulation
import pandas as pd

# Data Visualization
import matplotlib.pyplot as plt

# StatsModel
from statsmodels. datasets import get_rdataset

# SciPy
from scipy.cluster. hierarchy import (dendrogram, cut_tree)

# Scikit-Learn
from sklearn. decomposition import PCA
from sklearn. preprocessing import StandardScaler
from sklearn.cluster import (KMeans, AgglomerativeClustering)

# ISLP
from ISLP import load_data
from ISLP.cluster import compute_linkage

### Principal Components Analysis